# Applying Image Transformations With Bounding Boxes

At the core of this notebook we will use [imgaug](https://github.com/aleju/imgaug) library. Author has published [tutorials](https://nbviewer.org/github/aleju/imgaug-doc/tree/master/notebooks/) on the use of the library and [documentation](https://imgaug.readthedocs.io/en/latest/index.html) is available as well.

The inspiration for this document was taken from a tutorial available at [link](https://github.com/asetkn/Tutorial-Image-and-Multiple-Bounding-Boxes-Augmentation-for-Deep-Learning-in-4-Steps/blob/master/Tutorial-Image-and-Multiple-Bounding-Boxes-Augmentation-for-Deep-Learning-in-4-Steps.ipynb)

In [ ]:
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt

import imgaug as ia
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import augmenters as iaa
ia.seed(1)

import pandas as pd
import numpy as np
import re
import json
import io
import os
import boto3

In [ ]:
images_bucket_name = '<your-image-bucket>'
labelling_data_bucket_name = '<your-labelling-bucket>'

resized_image_path_prefix = 'resized'
transformed_image_path_prefix = 'transformed'
max_dimension = 416

labelling_data_filename = '<your-labelling-data>'
resized_data_bucket_path = f'{resized_image_path_prefix}/{labelling_data_filename}'
transformed_data_bucket_path = f'{transformed_image_path_prefix}/{labelling_data_filename}'
resized_data_filename = f'resized-{labelling_data_filename}'
transformed_data_filename = f'transformed-{labelling_data_filename}'

Let's write functions for importing and saving images and labelling data by key from and to S3. 

**NB!** Please note that this assumes the correct AWS region and credentials are set as default in your home `.aws/config` and `.aws/credentials` files.

In [ ]:
s3 = boto3.resource('s3')
img_bucket = s3.Bucket(images_bucket_name)
labels_bucket = s3.Bucket(labelling_data_bucket_name)

def get_image_from_bucket(key: str):
    object = img_bucket.Object(key)
    response_body = object.get()['Body']

    im = Image.open(response_body)

    return np.array(im)

def save_image_to_bucket(key: str, image_array):
    image_pil = Image.fromarray(image_array)
    
    image_buffer = io.BytesIO()
    image_pil.save(image_buffer, format='JPEG') 
    
    image_buffer.seek(0)
    
    img_bucket.upload_fileobj(image_buffer, key)

    image_buffer.close()
    
    print(f"Image uploaded to S3 bucket: {img_bucket.name}, key: {key}")

def get_resized_labelling_data():
    if os.path.isfile(resized_data_filename) is False:
        object = labels_bucket.Object(resized_data_bucket_path)
        content_bytes = object.get()['Body'].read()

        with open(resized_data_filename, 'wb') as local_file:
            local_file.write(content_bytes)

    return pd.read_csv(resized_data_filename)

def save_labelling_data_as_csv(bucket_key: str, filename: str, df):
    df.to_csv(filename, index=False, encoding='utf-8')
    labels_bucket.upload_file(filename, bucket_key)

    print(f"DataFrame uploaded to S3: {labels_bucket.name}, key: {bucket_key}")

Now we will read the csv file containing labelling data and transform it to a form of our liking.

We originally have the full S3 path, but we need just the key, so we will transform the `image` column to only hold the part after bucket name.

In [ ]:
labeldf = pd.read_csv(labelling_data_filename, usecols=['image', 'label'])

labeldf['image'] = labeldf['image'].str.replace(f's3://{images_bucket_name}/', '')

def transform_labels(labels):
    for label in labels:
        original_x = label['x']
        original_y = label['y']

        label['x'] = round(label['x'] * label['original_width'] / 100)
        label['y'] = round(label['y'] * label['original_height'] / 100)
        
        label['xMax'] = round((label['width'] + original_x) * label['original_width'] / 100)
        label.pop('width')
        label['yMax'] = round((label['height'] + original_y) * label['original_height'] / 100)
        label.pop('height')

        label['rectanglelabels'] = label['rectanglelabels'][0].split(' / ')[-1]

    return labels


## Resizing Images

In order to ensure that our GPU can handle using these images, and some of them can be quite large, we want to resize them together with their corresponding labels.

First step is to get the bounding boxes array from the label value dictionary that we have in our labels dataframe.

In [ ]:
class NumpyFloatValuesEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.float32):
            return float(obj)
        return json.JSONEncoder.default(self, obj)

def get_bounding_boxes_from_label_studio_labels(labels):
    return [
        BoundingBox(x1=l['x'], y1=l['y'], x2=l['xMax'], y2=l['yMax'], label=l['rectanglelabels'])
        for l in transform_labels(labels)
    ]

def get_bounding_boxes_from_labels(labels):
    return [
        BoundingBox(x1=l['x1'], y1=l['y1'], x2=l['x2'], y2=l['y2'], label=l['label'])
        for l in labels
    ]

def get_labels_from_bounding_boxes(bbs):
    return [
        {'x1': bbox.x1, 'y1': bbox.y1, 'x2': bbox.x2, 'y2': bbox.y2, 'label': bbox.label} 
        for bbox in bbs
    ]

Now we create resizers. These are used when an image height or width are larger than the given maximum.

In [ ]:
height_resize = iaa.Sequential([ 
    iaa.Resize({"height": max_dimension, "width": 'keep-aspect-ratio'})
])

width_resize = iaa.Sequential([ 
    iaa.Resize({"height": 'keep-aspect-ratio', "width": max_dimension})
])

Let's add a method to resize the image together with its bounding boxes and create a new dataframe with labels.

This will also populate the image bucket with the new resized images, at the path prefixed by the previously set `resized_image_path_prefix` value.

In [ ]:
def resize_imgaug(df):
    
    aug_img_label_array = [] 
    
    for index, row in df.iterrows():

        if (isinstance(row['label'], float)):
            image_labels = []
        else:
            image_labels = json.loads(row['label'])

        image_key = row['image']
        
        image = get_image_from_bucket(image_key)

        image_height = image.shape[0]
        image_width = image.shape[1]

        def resize_and_add_to_img_array(resize_function):
            bb_array = get_bounding_boxes_from_label_studio_labels(image_labels)
            
            bbs = BoundingBoxesOnImage(bb_array, shape=image.shape)

            #plt.imshow(bbs.draw_on_image(image, size=3))

            image_pad_function = iaa.Sequential([
                iaa.PadToFixedSize(width=max_dimension, height=max_dimension, position='center', pad_mode='constant', pad_cval=(166, 166))
            ])

            image_resize, bbs_resize = resize_function(image=image, bounding_boxes=bbs)
            image_aug, bbs_aug = image_pad_function(image=image_resize, bounding_boxes=bbs_resize)
            
            resized_image_key = f"{resized_image_path_prefix}/{image_key}"
            save_image_to_bucket(resized_image_key, image_aug)
            new_labels = get_labels_from_bounding_boxes(bbs_aug)
            img_label = {'image': resized_image_key, 'label': json.dumps(new_labels, cls=NumpyFloatValuesEncoder)}
            
            aug_img_label_array.append(img_label)

            # plt.imshow(bbs_aug.draw_on_image(image_aug, size=2))

        if image_height >= image_width and image_height > max_dimension:
            resize_and_add_to_img_array(height_resize)
            
        elif image_width > image_height and image_width > max_dimension:
            resize_and_add_to_img_array(width_resize)
        
        else:
            resize_and_add_to_img_array(lambda image, bounding_boxes : (image, bounding_boxes))
            
    return pd.DataFrame(aug_img_label_array) 

resize_df = resize_imgaug(labeldf)

Let's run this method and save the resulting dataframe with labelling data to a variable.

Then, we can use the previously defined method `save_labelling_data_as_csv` to add this as a csv file to the labelling bucket, to the path defined by variable `resized_data_bucket_path`. It will also be saved locally, to the current directory on your filesystem, with name defined by `resized_data_filename`.

In [ ]:
resize_df = resize_imgaug(labeldf)
save_labelling_data_as_csv(resized_data_bucket_path, resized_data_filename, resize_df)

## Transforming Images

The following chapter will apply augmentations to images and move the corresponding bounding boxes. Imgaug allows us to tune many cool parameters. Let's define a couple:

In [ ]:
aug = iaa.SomeOf(2, [    
    iaa.Affine(scale=(0.5, 2), cval=(166, 166)),
    iaa.Affine(rotate=(-60, 60), cval=(166, 166)),
    iaa.Affine(translate_percent={"x": (-0.3, 0.3), "y": (-0.3, 0.3)}, cval=(166, 166)),
    iaa.Fliplr(1),
    iaa.Flipud(1),
    iaa.Multiply((0.5, 1.5)),
    iaa.GaussianBlur(sigma=(1.0, 3.0)),
    iaa.AdditiveGaussianNoise(scale=(0.03*255, 0.05*255))
])

Next, we will define a function which would apply this augmentation to the images we just resized.

In [ ]:
def transform_imgaug(df, augmentor):
    
    aug_img_label_array = []
    
    for index, row in df.head(21).iterrows():

        image_key = row['image']
        image_labels = json.loads(row['label'])
        
        image = get_image_from_bucket(image_key)

        bb_array = get_bounding_boxes_from_labels(image_labels)
        bbs = BoundingBoxesOnImage(bb_array, shape=image.shape)

        for index in range(2):
            image_aug, bbs_aug = augmentor(image=image, bounding_boxes=bbs)
        
            bbs_aug = bbs_aug.remove_out_of_image()
            bbs_aug = bbs_aug.clip_out_of_image()

            # plt.imshow(bbs_aug.draw_on_image(image_aug, size=2))
            
            if re.findall('Image...', str(bbs_aug)) == ['Image([]']:
                new_labels = []
            else:
                new_labels = get_labels_from_bounding_boxes(bbs_aug)
            
            transformed_image_key = f"{transformed_image_path_prefix}/{image_key}-{index}.jpg"
            
            save_image_to_bucket(transformed_image_key, image_aug)
            img_label = {'image': transformed_image_key, 'label': json.dumps(new_labels, cls=NumpyFloatValuesEncoder)}
                
            aug_img_label_array.append(img_label)

    return pd.DataFrame(aug_img_label_array)       

Now, let's run this function to save the images to S3 and export labelling data to csv.

In [ ]:
resize_df = get_resized_labelling_data()
transform_df = transform_imgaug(resize_df, aug)
save_labelling_data_as_csv(transformed_data_bucket_path, transformed_data_filename, transform_df)